In [13]:
import os
from configparser import ConfigParser
path_config_file = os.getcwd().split('docrv2_sroie')[0] + 'docrv2_sroie/' + 'config.ini'
common_variable = ConfigParser()
common_variable.read(path_config_file)

print(common_variable.sections())
print(common_variable['database']['module_path'])
print(common_variable['database']['DB_BASE'])

import sys
sys.path.append(common_variable['database']['module_path'])
from db_util import *
from db import *

import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import time

from parse import parse
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from glob import glob
from tqdm import tqdm_notebook

['database', 'mlflow', 'font']
/media/myungsungkwak/msdisk/docrv2_sroie/modules/database/
/media/myungsungkwak/msdisk/docrv2_data/DB/


In [6]:
"""
Evalution (1)
"""
# raw db
raw_data_db_path = '/media/myungsungkwak/msdisk/docrv2_data/DB/raw_test_task12_img_txt_0001'
datadb = open_env(path=raw_data_db_path)
img_ref = read_bulk_data_from_db(datadb, prefix='ref')

# target db
target_db_name = "d2gan_inference_crop_0003_result_thr04"
target_db_path = os.path.join(common_variable['database']['DB_BASE'], target_db_name)
target_db = open_env(path=target_db_path)

img = read_bulk_data_from_db(target_db, prefix='img')
# img_c = read_bulk_data_from_db(datadb, prefix='img_c')
char_c = read_bulk_data_from_db(datadb, prefix='char_c')
# word_c = read_bulk_data_from_db(datadb, prefix='word_c')

In [7]:
"""
Evalution (2)
"""
output_path = '/media/myungsungkwak/msdisk/docrv2_data/d2gan_inference/'
output_path = os.path.join(output_path, target_db_name)
os.makedirs(output_path, exist_ok=True)

# create submission txt for each images
for idx in tqdm(range(len(img_ref))):
    tchar = char_c[idx]
    for ii in range(len(tchar)):
        tchar[ii] = [float (i) for i in tchar[ii][0:8]]
        tchar[ii] = [int (i) for i in tchar[ii][0:8]]
    
    txt_filename = img_ref[idx]+'.txt'
    df = pd.DataFrame(tchar)
    df.to_csv(os.path.join(output_path,txt_filename),sep=',',index=False,header=False)

100%|██████████| 10/10 [00:00<00:00, 326.01it/s]


In [31]:
"""
Evaluation (3)
"""
zip_name = 'submit'
shutil.make_archive(zip_name, 'zip', output_path) # to avoid including submit.zip

# move submit.zip to task evaluation path
eval_path = '/media/myungsungkwak/msdisk/docrv2_sroie/evaluation/task1/'
try:
    shutil.move(os.path.join(os.getcwd(), zip_name+".zip"), eval_path) # move zip to output dir
except:
    now = time.localtime()
    change_zip_name = zip_name+"{}{}{}".format(str(now.tm_hour), str(now.tm_min), str(now.tm_sec))
    os.rename(os.path.join(os.getcwd(), zip_name+".zip"),
              os.path.join(os.getcwd(), change_zip_name+'.zip')
             )
    zip_name = change_zip_name
    shutil.move(os.path.join(os.getcwd(), zip_name+".zip"), eval_path) # move zip to output dir

# copy gt.zip to task evaluation path
gt_path = '/media/myungsungkwak/msdisk/docrv2_data/d2gan_inference/gt/'
gt_name = "gt.zip"
shutil.copy(gt_path+gt_name, eval_path)

print("Run command in {}\n$ python script.py -g=gt.zip -s={}.zip".format(eval_path, zip_name))

Run command in /media/myungsungkwak/msdisk/docrv2_sroie/evaluation/task1/
$ python script.py -g=gt.zip -s=submit1400.zip
